In [1]:
import sys

assert sys.version_info >= (3, 7)
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

import numpy as np 

2024-09-05 14:22:13.978444: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 14:22:13.978522: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 14:22:14.089565: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-05 14:22:14.323926: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 14:22:16.182380: W tensorflow/compiler/tf2

In [2]:
# read the dataset
with open("cleaned_combined_Elytis.txt") as f:
    data = f.read() 

def clean_text(text):
    import re 
    """
    Removes all non-Greek characters and numbers from the given text.

    Args:
    text (str): The input text to process.

    Returns:
    str: The cleaned text containing only Greek characters.
    """
    # Define a regular expression pattern that matches Greek characters (uppercase and lowercase)
    # and spaces (optional if you want to preserve spaces)
    pattern = r'[^\u0370-\u03FF\u1F00-\u1FFF\s]'
    
    # Use re.sub to replace all characters that don't match the pattern with an empty string
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text
data = clean_text(data) 
n_distinct_characters = len(set(data)) 


text_vec_layer = tf.keras.layers.TextVectorization(
    split="character", standardize="lower")
text_vec_layer.adapt([data])
encoded = text_vec_layer([data])[0]
encoded -= 2  # drop tokens 0 (pad) and 1 (unknown), which we will not use
n_tokens = text_vec_layer.vocabulary_size() - 2 # number of distinct chars = 39
dataset_size = len(encoded)

print(n_tokens) 

2024-09-05 14:22:29.322297: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


37


In [3]:
# encoded[:1_000_000]
len(data)

489572

In [4]:
length = 100
tf.random.set_seed(42)

# Total length of the dataset
total_length = len(data) 

# Adjust the splits accordingly
train_end = int(0.8 * total_length)  # 80% for training
valid_end = int(0.9 * total_length)  # 10% for validation, 10% for testing

In [5]:
def to_dataset_for_stateful_rnn(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length + 1)).batch(1)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

stateful_train_set = to_dataset_for_stateful_rnn(encoded[:1_000_000], length)
stateful_valid_set = to_dataset_for_stateful_rnn(encoded[1_000_000:1_060_000],
                                                 length)
stateful_test_set = to_dataset_for_stateful_rnn(encoded[1_060_000:], length)

In [6]:
def save_datasets():
    # Save datasets to disk
    tf.data.experimental.save(stateful_train_set, "stateful_train_set")
    tf.data.experimental.save(stateful_valid_set, "stateful_valid_set")
    tf.data.experimental.save(stateful_test_set, "stateful_test_set")

    print("Datasets saved successfully.") 
# save_datasets() 

In [7]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16,
                              batch_input_shape=[1, None]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [8]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [9]:
# extra code – use a different directory to save the checkpoints
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_stateful_Elytis_model",
    monitor="val_accuracy",
    save_best_only=True)

In [10]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
# Perform a single evaluation step to build the metrics
model.evaluate(stateful_valid_set, verbose=0)
history = model.fit(stateful_train_set, validation_data=stateful_valid_set,
                    epochs=10, callbacks=[ResetStatesCallback(), model_ckpt])

Epoch 1/10
   4893/Unknown - 128s 26ms/step - loss: 2.2507 - accuracy: 0.3421WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.
4895/4895 [==============================] - 129s 26ms/step - loss: 2.2505 - accuracy: 0.3421
Epoch 2/10
4895/4895 [==============================] - 126s 26ms/step - loss: 1.9178 - accuracy: 0.4303
Epoch 3/10
4895/4895 [==============================] - 126s 26ms/step - loss: 1.8020 - accuracy: 0.4649
Epoch 4/10
4895/4895 [==============================] - 124s 25ms/step - loss: 1.7370 - accuracy: 0.4836
Epoch 5/10
4895/4895 [==============================] - 126s 26ms/step - loss: 1.6946 - accuracy: 0.4964
Epoch 6/10
4895/4895 [==============================] - 139s 28ms/step - loss: 1.6647 - accuracy: 0.5047
Epoch 7/10
4895/4895 [==============================] - 124s 25ms/step - loss: 1.6424 - accuracy: 0.5111
Epoch 8/10
4895/4895 [==============================] - 127s 26ms/step - loss: 1.6249 - accuracy: 0.5159
Epoch 9/10


In [11]:
stateless_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [12]:
stateless_model.build(tf.TensorShape([None, None]))

In [13]:
stateless_model.set_weights(model.get_weights())

In [14]:
Elytis_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    stateless_model
])

In [15]:
# Save the model in HDF5 format
# Elytis_model.save('elytis_stateful_model')

In [16]:
# Elytis_model = tf.keras.models.load_model('Elytis_stateful_model')

In [17]:
def next_char(text, temperature=1):
    y_proba = Elytis_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [21]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [27]:
# results = {}
# for i in np.arange(.1, 1.2, 0.1):
#     text = extend_text("καλώς εχόντων τω", 1000, i) 
#     results[i] = text
# print(results) 

1/1 [==============================] - 0s 72ms/step
{0.1: 'καλώς εχόντων των αντρών με το καλοκαίρι στο στήθος το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού τη στα ματιά της αποστήθες και το παιδί με το παντού το παιδί με το παντού το παιδί με το κατά το παντού το παιδί με το καλοκαίρι μες στο στήθος το παιδί με το παντού το παιδί με το καλοκαίρι μες στο μαντικά του παντού με το παντού το περιμένο μες στο στήθος το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού τα περιστέρια με το παντού το παιδί με το καλοκαίρι μες στο στήθος το παιδί με το παντού τα παιδιά του παντού μες στο στήθος το παιδί με το καλοκαίρι μες στο στήθος το παιδί με το παντού το παιδί με το παντού το παιδί με το κατά το παιδί μες στα σκοτεινά τη στα ματιά του παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού

In [36]:
print(results) 

{0.1: 'καλώς εχόντων των αντρών με το καλοκαίρι στο στήθος το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού τη στα ματιά της αποστήθες και το παιδί με το παντού το παιδί με το παντού το παιδί με το κατά το παντού το παιδί με το καλοκαίρι μες στο στήθος το παιδί με το παντού το παιδί με το καλοκαίρι μες στο μαντικά του παντού με το παντού το περιμένο μες στο στήθος το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού τα περιστέρια με το παντού το παιδί με το καλοκαίρι μες στο στήθος το παιδί με το παντού τα παιδιά του παντού μες στο στήθος το παιδί με το καλοκαίρι μες στο στήθος το παιδί με το παντού το παιδί με το παντού το παιδί με το κατά το παιδί μες στα σκοτεινά τη στα ματιά του παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το παντού το παιδί με το περιμόνι το πα